In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
driver = webdriver.Chrome()

In [ ]:
search_query = input("Manually search for a product on Amazon. Press Enter after completing the search.")

url = ""
driver.get(url)

# Wait for the dynamic content to load (adjust the time as needed)
driver.implicitly_wait(10)

In [ ]:
product_elements = driver.find_elements(By.CSS_SELECTOR, 'div.a-section.a-spacing-base')

In [ ]:
def extract_product_info(product_element):
    try:
        # Extract title
        title_element = product_element.find_element(By.CSS_SELECTOR, 'h2.a-size-mini.a-spacing-none.a-color-base.s-line-clamp-4')
        title = title_element.text.strip()

        # Extract price
        price = product_element.find_element(By.CSS_SELECTOR, 'span.a-price-whole').text.strip() + "." + product_element.find_element(By.CSS_SELECTOR, 'span.a-price-fraction').text.strip()

        # Return the title and price as a tuple
        return (title, price)
    except NoSuchElementException as e:
        # Handle the exception by printing a message
        print("Error occurred while extracting product information:", e)
        return None

In [ ]:
products_info = []

# Use ThreadPoolExecutor to execute tasks in parallel using 4 cores
with ThreadPoolExecutor(max_workers=4) as executor:
    # Submit the scraping tasks and get the futures
    futures = [executor.submit(extract_product_info, product_element) for product_element in product_elements]

    # Wait for all tasks to complete and process the results
    for future in as_completed(futures):
        result = future.result()
        if result is not None:
            products_info.append(result)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({"name": [n[0] for n in products_info], "price": [n[1] for n in products_info]})

In [ ]:
df

**EDA**

In [ ]:
df.max()